In [2]:
import os
import shutil

import matplotlib.pyplot as plt

import nibabel as nib
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from scipy.spatial import ConvexHull
from skimage import measure

import glob
import csv
import cv2
import boto3
import wget

import s3fs
import fsspec

from io import BytesIO
from nibabel import FileHolder, Nifti1Image, Nifti1Header

import boto3
import io
import pickle

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = pd.read_csv('s3://*?????*/lab2/metadata/clinicaldata.csv')
df['image_id'] = df.image_id.str.strip()
df = df[df.type.isin([0, 2])]
df = df.rename({'type': 'has_covid'}, axis=1)
df.loc[df.has_covid == 2, 'has_covid'] = 1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.has_covid.value_counts().plot.bar();
plt.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def show_slice(slice):
    plt.figure()
    plt.imshow(slice.T, cmap="gray", origin="lower")

def show_slice_window(slice, level, window):
    max = level + window/2
    min = level - window/2
    slice = slice.clip(min,max)
    plt.figure()
    plt.imshow(slice.T, cmap="gray", origin="lower")
    plt.savefig('L'+str(level)+'W'+str(window))


def overlay_plot(im, mask):
    plt.figure()
    plt.imshow(im.T, 'gray', interpolation='none')
    plt.imshow(mask.T, 'jet', interpolation='none', alpha=0.5)
    
def make_dirs(path):
    if os.path.exists(path):
        shutil.rmtree(path)
        os.mkdir(path)
    else:
        os.makedirs(path)

def intensity_seg(ct_numpy, min, max):
    clipped = ct_numpy.clip(min, max)
    clipped[clipped != max] = 1
    clipped[clipped == max] = 0
    return measure.find_contours(clipped, 0.95)

def set_is_closed(contour):
    if contour_distance(contour) < 1:
        return True
    else:
        return False

def find_lungs(contours):
    body_and_lung_contours = []
    vol_contours = []

    for contour in contours:
        hull = ConvexHull(contour)

        if hull.volume > 2000 and set_is_closed(contour):
            body_and_lung_contours.append(contour)
            vol_contours.append(hull.volume)

    if len(body_and_lung_contours) == 2:
        return body_and_lung_contours
    elif len(body_and_lung_contours) > 2:
        (vol_contours, 
         body_and_lung_contours) = (list(t) 
                                    for t in 
                                    zip(*sorted(zip(vol_contours, 
                                                    body_and_lung_contours))))
        body_and_lung_contours.pop(-1)
        return body_and_lung_contours


def show_contour(image, contours, name=None, save=False):
    fig, ax = plt.subplots()
    ax.imshow(image.T, cmap=plt.cm.gray)
    for contour in contours:
        ax.plot(contour[:, 0], contour[:, 1], linewidth=1)

    ax.set_xticks([])
    ax.set_yticks([])

    if save:
        plt.savefig(name)
        plt.close(fig)
    else:
        plt.show()



def create_mask_from_polygon(image, contours):
    lung_mask = np.array(Image.new('L', image.shape, 0))
    mask_check = []
    for contour in contours:
        x = contour[:, 0]
        y = contour[:, 1]
        polygon_tuple = list(zip(x, y))
        img = Image.new('L', image.shape, 0)
        ImageDraw.Draw(img).polygon(polygon_tuple, outline=0, fill=1)
        mask = np.array(img)
        if mask.sum() > 2000:
            mask_check.append(mask)
            lung_mask += mask

    if (lung_mask.max() > 1) or (len(mask_check) < 2):
        return np.array([])
    else:
        return lung_mask.T

def save_nifty(img_np, name, affine):
    img_np[img_np == 1] = 255
    ni_img = nib.Nifti1Image(img_np, affine)
    nib.save(ni_img, name + '.nii')


def find_pix_dim(ct_img):
    pix_dim = ct_img.header["pixdim"]
    dim = ct_img.header["dim"]
    max_indx = np.argmax(dim)
    pixdimX = pix_dim[max_indx]
    dim = np.delete(dim, max_indx)
    pix_dim = np.delete(pix_dim, max_indx)
    max_indy = np.argmax(dim)
    pixdimY = pix_dim[max_indy]
    return [pixdimX, pixdimY]

def contour_distance(contour):
    dx = contour[0, 1] - contour[-1, 1]
    dy = contour[0, 0] - contour[-1, 0]
    return np.sqrt(np.power(dx, 2) + np.power(dy, 2))

def compute_area(mask, pixdim):
    mask[mask >= 1] = 1
    lung_pixels = np.sum(mask)
    return lung_pixels * pixdim[0] * pixdim[1]

def euclidean_dist(dx, dy):
    return np.sqrt(np.power(dx, 2) + np.power(dy, 2))

def denoise_vessels(lung_contour, vessels):
    vessels_coords_x, vessels_coords_y = np.nonzero(vessels)
    for contour in lung_contour:
        x_points, y_points = contour[:, 0], contour[:, 1]
        for (coord_x, coord_y) in zip(vessels_coords_x, vessels_coords_y):
            for (x, y) in zip(x_points, y_points):
                d = euclidean_dist(x - coord_x, y - coord_y)
                if d <= 0.1:
                    vessels[coord_x, coord_y] = 0
    return vessels

def split_array_coords(array, indx=0, indy=1):
    x = [array[i][indx] for i in range(len(array))]
    y = [array[i][indy] for i in range(len(array))]
    return x, y


def create_vessel_mask(lung_mask, ct_numpy, denoise=False):
    vessels = lung_mask * ct_numpy
    vessels[vessels == 0] = -1000
    vessels[vessels >= -500] = 1
    vessels[vessels < -500] = 0
    show_slice(vessels)
    if denoise:
        return denoise_vessels(lungs, vessels)
    
    show_slice(vessels)

    return vessels

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def filter_slices(exam_path):
    try:
        s3_client = boto3.client('s3')
        s3_response_object = s3_client.get_object(Bucket='*?????????*', 
                                                  Key=exam_path)

        object_content = s3_response_object['Body'].read()


        fh = FileHolder(fileobj=BytesIO(object_content))
        img = Nifti1Image.from_file_map({'header': fh, 'image': fh})

        img_name = exam_path.split("/")[-1].split('.nii')[0]

        ct_img = img
        pixdim = find_pix_dim(ct_img)

        lung_list = []
        for i in range(ct_img.shape[2]):
            ct_numpy = ct_img.get_fdata()[:, :, i]

            contours = intensity_seg(ct_numpy, min=-1000, max=-300)
            lungs = find_lungs(contours)

            if lungs == None:
                lung_area = 0
                lung_list.append(lung_area)
            else:
                lung_mask = create_mask_from_polygon(ct_numpy, lungs)

                if not lung_mask.any():
                    lung_area = 0
                    lung_list.append(lung_area)

                else:
                    lung_area = compute_area(lung_mask, find_pix_dim(ct_img))
                    lung_list.append(lung_area)

        ct_numpy = ct_img.get_fdata()[:, :, np.argmax(lung_list)] 

        contours = intensity_seg(ct_numpy, min=-1000, max=-300)

        lungs = find_lungs(contours)
        lung_mask = create_mask_from_polygon(ct_numpy, lungs)
        img = cv2.rotate(ct_numpy, cv2.ROTATE_90_COUNTERCLOCKWISE)
    except:
        return None
    
    return ct_numpy

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
s = 'lab2/images/' + df.image_id + '.nii'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
x = (sc.parallelize(s.to_list()[:500])
       .map(lambda x: (x.split("/")[-1].split('.nii')[0], 
                       filter_slices(x)))).collect()

s3_client = boto3.client('s3')
my_array_data = io.BytesIO()
pickle.dump(x, my_array_data)
my_array_data.seek(0)
s3_client.upload_fileobj(my_array_data, '*??????*', 'data4.pkl')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
x = (sc.parallelize(s.to_list()[500:])
       .map(lambda x: (x.split("/")[-1].split('.nii')[0], 
                       filter_slices(x)))).collect()

s3_client = boto3.client('s3')
my_array_data = io.BytesIO()
pickle.dump(x, my_array_data)
my_array_data.seek(0)
s3_client.upload_fileobj(my_array_data, '*????????*', 'data3.pkl')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…